# RNAqa descriptors output analysis

In [1]:
!pip3 install xmltodict biopython tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.6 MB/s eta 0:00:00:00:0100:01


In [48]:
import xmltodict
import glob
import os
import traceback
import pandas as pd
import tempfile
from tqdm import tqdm
from Bio.PDB import PDBParser
from Bio.PDB import PDBIO
class Structure:
    rmsd=[]
    residues=0
    chains=[]
    name=''
    error=0
    def __hash__(self):
        return self.i
    def __eq__(self, other):
        if isinstance(other, Structure):
            return self.i == other.i
        return False
    def get_df(self):
        return pd.DataFrame(data={'structure_index':self.index,
                           'structure_name':self.name,
                           'rmsd':self.rmsd,
                           'segments_no':self.segments_no,
                           'elements_no':self.elements_no,
                           'residues_no':self.residues_no,
                           'path':self._path,
                           'range1':self.range1,
                           'range2':self.range2
                          }
                           )
    def __init__(self,path_to_parent,index):
        self.index=[]
        self.rmsd=[]
        self.name=[]
        self.segments_no=[]
        self.elements_no=[]
        self.residues_no=[]
        self.range1=[]
        self.range2=[]
        self._path=[]
        result_df=pd.DataFrame()
        with tempfile.NamedTemporaryFile() as fp:
            with open(os.path.join(path_to_parent,'descs','results.txt'),'r') as enter:
                fp.write("1\t2\t3\t4\t5\t6\t7\n".encode('ascii'))
                fp.write(''.join(enter.readlines()[21:-1]).encode('ascii'))
                fp.seek(0)
            result_df = pd.read_csv(fp.name,sep='\t',index_col=False)
            result_df.columns=["ResNo",   "Id",      "SegmentsNo",      "ElementsNo",      "ResiduesNo", "range1","range2"]
        xmls = glob.glob(os.path.join(path_to_parent,'descs','*_beauty.xml'))
        for xml in xmls:
            dir_name=xml.replace('_beauty.xml','')
            with open(xml) as f:
                for i in xmltodict.parse(f.read())['measureScores']['structure']:
                    self.rmsd.append(float(i['score']))
                    file_path = os.path.join(dir_name,i['description']['filename'])
                    if not os.path.exists(file_path):
                        raise Exception('File not exists ',file_path)
                    
                    self.name.append('_'.join(i['description']['filename'].split('_')[:-1]))
                    self._path.append('/'.join(os.path.join(dir_name,i['description']['filename']).split('/')[-2:]))
                    serie=result_df[result_df.Id==dir_name.split('/')[-1]]

                    self.segments_no.append(serie.iloc[0]['SegmentsNo'])
                    self.elements_no.append(serie.iloc[0]['ElementsNo'])
                    self.residues_no.append(serie.iloc[0]['ResiduesNo'])
                    self.range1.append(serie.iloc[0]['range1'])
                    self.range2.append(serie.iloc[0]['range2'])
                    self.index.append(index)
                    
directories=tqdm(glob.glob("/local/tmp/ts/preds/*"))
result_df=pd.DataFrame({'structure_index':[],
                           'structure_name':[],
                           'rmsd':[],
                           'segments_no':[],
                           'elements_no':[],
                           'residues_no':[],
                           'path':[],
                           'range1':[],
                           'range2':[]
                       })
for i,path_to_parent in enumerate(directories):
    try:
        s=Structure(path_to_parent,i)
        result_df=pd.concat([result_df,s.get_df()], ignore_index=True) 
    except Exception:
        print(traceback.format_exc())
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 737/737 [24:42<00:00,  2.01s/it]


[2, 3]

In [54]:
result_df[result_df.rmsd<=0]

,structure_index,structure_name,rmsd,segments_no,elements_no,residues_no,path,range1,range2
102725,84.0,1MFQ_1_A,-1.0,3.0,7.0,24.0,rs_1MFQ_1_A_A_171_U/1MFQ_1_A_27.pdb,"A126-A132, A166-A176, A222-A227","CAUCAAU, UUGCCUAAGGA, GUGCUG"
102726,84.0,1MFQ_1_A,-1.0,3.0,7.0,24.0,rs_1MFQ_1_A_A_171_U/1MFQ_1_A_13.pdb,"A126-A132, A166-A176, A222-A227","CAUCAAU, UUGCCUAAGGA, GUGCUG"
102727,84.0,1MFQ_1_A,-1.0,3.0,7.0,24.0,rs_1MFQ_1_A_A_171_U/1MFQ_1_A_7.pdb,"A126-A132, A166-A176, A222-A227","CAUCAAU, UUGCCUAAGGA, GUGCUG"
102728,84.0,1MFQ_1_A,-1.0,3.0,7.0,24.0,rs_1MFQ_1_A_A_171_U/1MFQ_1_A_14.pdb,"A126-A132, A166-A176, A222-A227","CAUCAAU, UUGCCUAAGGA, GUGCUG"
102729,84.0,1MFQ_1_A,-1.0,3.0,7.0,24.0,rs_1MFQ_1_A_A_171_U/1MFQ_1_A_18.pdb,"A126-A132, A166-A176, A222-A227","CAUCAAU, UUGCCUAAGGA, GUGCUG"
...,...,...,...,...,...,...,...,...,...
104054,84.0,1MFQ_1_A,-1.0,2.0,6.0,22.0,rs_1MFQ_1_A_A_169_C/1MFQ_1_A_10.pdb,"A127-A134, A164-A177","AUCAAUAU, GGUUGCCUAAGGAG"
104055,84.0,1MFQ_1_A,-1.0,2.0,6.0,22.0,rs_1MFQ_1_A_A_169_C/1MFQ_1_A_17.pdb,"A127-A134, A164-A177","AUCAAUAU, GGUUGCCUAAGGAG"
104056,84.0,1MFQ_1_A,-1.0,2.0,6.0,22.0,rs_1MFQ_1_A_A_169_C/1MFQ_1_A_20.pdb,"A127-A134, A164-A177","AUCAAUAU, GGUUGCCUAAGGAG"
104057,84.0,1MFQ_1_A,-1.0,2.0,6.0,22.0,rs_1MFQ_1_A_A_169_C/1MFQ_1_A_11.pdb,"A127-A134, A164-A177","AUCAAUAU, GGUUGCCUAAGGAG"


In [108]:
test_df = pd.read_csv('./test.csv', index_col=False)
val_df = pd.read_csv('./val.csv', index_col=False)
train_df = pd.read_csv('./train.csv', index_col=False)

In [115]:
new_test_df=result_df[result_df.structure_name.isin(['_'.join(i.replace('.pdb','').split('_')[:-1]) for i in list(set(test_df['description'].values))])]

In [116]:
new_val_df=result_df[result_df.structure_name.isin(['_'.join(i.replace('.pdb','').split('_')[:-1]) for i in list(set(val_df['description'].values))])]

In [117]:
new_train_df=result_df[result_df.structure_name.isin(['_'.join(i.replace('.pdb','').split('_')[:-1]) for i in list(set(train_df['description'].values))])]

In [119]:
new_train_df

,structure_index,structure_name,rmsd,segments_no,elements_no,residues_no,path,range1,range2
0,0.0,5ZW4_1_D,3.365,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_28.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG"
1,0.0,5ZW4_1_D,3.317,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_15.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG"
2,0.0,5ZW4_1_D,3.317,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_27.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG"
3,0.0,5ZW4_1_D,2.790,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_2.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG"
4,0.0,5ZW4_1_D,3.364,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_3.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG"
...,...,...,...,...,...,...,...,...,...
909037,736.0,7QI6_1_Ax,17.783,4.0,8.0,33.0,rs_7QI6_1_Ax_A_11_U/7QI6_1_Ax_11.pdb,"A5-A16, A19-A25, A39-A47, A61-A65","GAAUAGUUUAAC, AACUCCU, GAGAUGGAG, UUCCC"
909038,736.0,7QI6_1_Ax,16.933,4.0,8.0,33.0,rs_7QI6_1_Ax_A_11_U/7QI6_1_Ax_20.pdb,"A5-A16, A19-A25, A39-A47, A61-A65","GAAUAGUUUAAC, AACUCCU, GAGAUGGAG, UUCCC"
909039,736.0,7QI6_1_Ax,16.021,4.0,8.0,33.0,rs_7QI6_1_Ax_A_11_U/7QI6_1_Ax_22.pdb,"A5-A16, A19-A25, A39-A47, A61-A65","GAAUAGUUUAAC, AACUCCU, GAGAUGGAG, UUCCC"
909040,736.0,7QI6_1_Ax,16.509,4.0,8.0,33.0,rs_7QI6_1_Ax_A_11_U/7QI6_1_Ax_26.pdb,"A5-A16, A19-A25, A39-A47, A61-A65","GAAUAGUUUAAC, AACUCCU, GAGAUGGAG, UUCCC"


,structure_index,structure_name,rmsd,segments_no,elements_no,residues_no,path,range1,range2


In [120]:
new_train_df[new_train_df.structure_name=='1MFQ_1_A']

,structure_index,structure_name,rmsd,segments_no,elements_no,residues_no,path,range1,range2
102667,84.0,1MFQ_1_A,0.681,3.0,8.0,33.0,rs_1MFQ_1_A_A_128_U/1MFQ_1_A_27.pdb,"A123-A132, A169-A178, A216-A228","CGGCAUCAAU, CCUAAGGAGG, ACUCCCGUGCUGA"
102668,84.0,1MFQ_1_A,0.822,3.0,8.0,33.0,rs_1MFQ_1_A_A_128_U/1MFQ_1_A_13.pdb,"A123-A132, A169-A178, A216-A228","CGGCAUCAAU, CCUAAGGAGG, ACUCCCGUGCUGA"
102669,84.0,1MFQ_1_A,0.637,3.0,8.0,33.0,rs_1MFQ_1_A_A_128_U/1MFQ_1_A_7.pdb,"A123-A132, A169-A178, A216-A228","CGGCAUCAAU, CCUAAGGAGG, ACUCCCGUGCUGA"
102670,84.0,1MFQ_1_A,0.800,3.0,8.0,33.0,rs_1MFQ_1_A_A_128_U/1MFQ_1_A_14.pdb,"A123-A132, A169-A178, A216-A228","CGGCAUCAAU, CCUAAGGAGG, ACUCCCGUGCUGA"
102671,84.0,1MFQ_1_A,0.671,3.0,8.0,33.0,rs_1MFQ_1_A_A_128_U/1MFQ_1_A_18.pdb,"A123-A132, A169-A178, A216-A228","CGGCAUCAAU, CCUAAGGAGG, ACUCCCGUGCUGA"
...,...,...,...,...,...,...,...,...,...
103996,84.0,1MFQ_1_A,0.684,3.0,7.0,23.0,rs_1MFQ_1_A_A_120_G/1MFQ_1_A_10.pdb,"A116-A124, A220-A225, A229-A236","CUAAGUUCG, CCGUGC, UCAGUAGU"
103997,84.0,1MFQ_1_A,0.860,3.0,7.0,23.0,rs_1MFQ_1_A_A_120_G/1MFQ_1_A_17.pdb,"A116-A124, A220-A225, A229-A236","CUAAGUUCG, CCGUGC, UCAGUAGU"
103998,84.0,1MFQ_1_A,0.754,3.0,7.0,23.0,rs_1MFQ_1_A_A_120_G/1MFQ_1_A_20.pdb,"A116-A124, A220-A225, A229-A236","CUAAGUUCG, CCGUGC, UCAGUAGU"
103999,84.0,1MFQ_1_A,0.666,3.0,7.0,23.0,rs_1MFQ_1_A_A_120_G/1MFQ_1_A_11.pdb,"A116-A124, A220-A225, A229-A236","CUAAGUUCG, CCGUGC, UCAGUAGU"


In [122]:
new_test_df["file"] = new_test_df["path"].apply(lambda x: x.replace('/','_'))
new_val_df["file"] = new_val_df["path"].apply(lambda x: x.replace('/','_'))
new_train_df["file"] = new_train_df["path"].apply(lambda x: x.replace('/','_'))

/tmp/ipykernel_1189/3155376374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df["file"] = new_test_df["path"].apply(lambda x: x.replace('/','_'))
/tmp/ipykernel_1189/3155376374.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_val_df["file"] = new_val_df["path"].apply(lambda x: x.replace('/','_'))
/tmp/ipykernel_1189/3155376374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [124]:
new_test_df.to_csv('./test.csv')
new_val_df.to_csv('./val.csv')
new_train_df.to_csv('./train.csv')

In [123]:
new_train_df.head

,structure_index,structure_name,rmsd,segments_no,elements_no,residues_no,path,range1,range2,file
0,0.0,5ZW4_1_D,3.365,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_28.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG",rs_5ZW4_1_D_A_14_A_5ZW4_1_D_28.pdb
1,0.0,5ZW4_1_D,3.317,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_15.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG",rs_5ZW4_1_D_A_14_A_5ZW4_1_D_15.pdb
2,0.0,5ZW4_1_D,3.317,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_27.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG",rs_5ZW4_1_D_A_14_A_5ZW4_1_D_27.pdb
3,0.0,5ZW4_1_D,2.790,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_2.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG",rs_5ZW4_1_D_A_14_A_5ZW4_1_D_2.pdb
4,0.0,5ZW4_1_D,3.364,2.0,4.0,13.0,rs_5ZW4_1_D_A_14_A/5ZW4_1_D_3.pdb,"A2-A8, A11-A16","CUGCUUU, CGCAGG",rs_5ZW4_1_D_A_14_A_5ZW4_1_D_3.pdb
...,...,...,...,...,...,...,...,...,...,...
909037,736.0,7QI6_1_Ax,17.783,4.0,8.0,33.0,rs_7QI6_1_Ax_A_11_U/7QI6_1_Ax_11.pdb,"A5-A16, A19-A25, A39-A47, A61-A65","GAAUAGUUUAAC, AACUCCU, GAGAUGGAG, UUCCC",rs_7QI6_1_Ax_A_11_U_7QI6_1_Ax_11.pdb
909038,736.0,7QI6_1_Ax,16.933,4.0,8.0,33.0,rs_7QI6_1_Ax_A_11_U/7QI6_1_Ax_20.pdb,"A5-A16, A19-A25, A39-A47, A61-A65","GAAUAGUUUAAC, AACUCCU, GAGAUGGAG, UUCCC",rs_7QI6_1_Ax_A_11_U_7QI6_1_Ax_20.pdb
909039,736.0,7QI6_1_Ax,16.021,4.0,8.0,33.0,rs_7QI6_1_Ax_A_11_U/7QI6_1_Ax_22.pdb,"A5-A16, A19-A25, A39-A47, A61-A65","GAAUAGUUUAAC, AACUCCU, GAGAUGGAG, UUCCC",rs_7QI6_1_Ax_A_11_U_7QI6_1_Ax_22.pdb
909040,736.0,7QI6_1_Ax,16.509,4.0,8.0,33.0,rs_7QI6_1_Ax_A_11_U/7QI6_1_Ax_26.pdb,"A5-A16, A19-A25, A39-A47, A61-A65","GAAUAGUUUAAC, AACUCCU, GAGAUGGAG, UUCCC",rs_7QI6_1_Ax_A_11_U_7QI6_1_Ax_26.pdb
